In [1]:
from board3 import Board3
from controller3 import ActionController

import random

import torch
import torch.nn as nn
import torch.optim as optim

from torch.distributions.categorical import Categorical

from tqdm.auto import tqdm

In [2]:
def to_emb(b: Board3):

    py, px = b.get_player_position()
    ey, ex = b.get_enemy_position()
    ty, tx = b.get_todd_position()

    mws = [0 for _ in range(16)]

    for my, mx in b.mw:
        mws[my * 4 + mx] = 1

    mwe = torch.Tensor(mws)

    pe = torch.cat([nn.functional.one_hot(torch.LongTensor([x]), num_classes=4) for x in [py, px, ey, ex, ty, tx]], dim=0)

    return torch.cat([pe.flatten().float(), mwe], dim=0).unsqueeze(0)

In [8]:
b = Board3(walk_frodo=False, walk_time=1000)
b.players_positions = [(2, 3), (2, 2), (3, 3)]

print(b)

.  %  .  %
.  %  .  %
.  .  E  P
.  .  .  T


In [13]:


def gen_new_board():
    b = Board3(walk_frodo=False, walk_time=1000)
    b.players_positions = [(2, 3), (2, 2), (3, 3)]
    return b

ag = {}

for e in tqdm(range(10000)):

    b = gen_new_board()

    h = []

    for _ in range(50):

        if b not in ag:
            ag[b] = [0.5 for _ in range(ActionController.get_action_space())]

        if random.random() < 0.1:
            ra = random.randint(0, ActionController.get_action_space() - 1)
        else:
            ra = max(list(range(ActionController.get_action_space())), key=lambda x: ag[b][x])

        bc, r, end = ActionController.step(b, ra, 1600)

        if bc not in ag:
            ag[bc] = [0.5 for _ in range(ActionController.get_action_space())]

        if end:
            y = r
        else:
            y = r - ag[bc][ra] + 0.9 * max(ag[bc])

        ag[b][ra] += 0.1 * y

        if end:
            break

        b = bc




  0%|          | 0/10000 [00:00<?, ?it/s]

In [56]:
b = gen_new_board()
b.set_enemy(0, 2)
print(b)
if b not in ag:
    ag[b] = [0.5 for _ in range(ActionController.get_action_space())]

a = max(list(range(ActionController.get_action_space())), key=lambda x: ag[b][x])
print(a)
print(ActionController.explain_action(a))

.  %  E  %
.  %  .  %
.  .  .  P
.  .  .  T
29
MW 1 2


In [14]:
def eval_nt(ag, n = 100, e = 50, time_step=1600):

    w, d, l = 0, 0, 0

    for _ in range(n):
        b = gen_new_board()

        for _ in range(e):
            if b not in ag:
                ag[b] = [0.5 for _ in range(ActionController.get_action_space())]

            a = max(list(range(ActionController.get_action_space())), key=lambda x: ag[b][x])
            bc, r, end = ActionController.step(b, a, time_step)
            if end:
                if r == 1:
                    w += 1
                elif r == -1:
                    l += 1
                break
            b = bc
        else:
            d += 1


    return w / n, d / n, l / n


eval_nt(ag)



(0.0, 1.0, 0.0)